# Process Individual ROH csv files into Summary Files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import gridspec
import socket
import os as os
import sys as sys
import multiprocessing as mp
import matplotlib.colors as cls
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)

sys.path.append("./Python3/")  # Since now we are in the Root Directory
sys.path.append("./PackagesSupport/")
from pp_individual_roh_csvs import create_combined_ROH_df, give_iid_paths, pp_individual_roh

print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

midway2-0402.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


# Eigenstrat ROH Results postprocessing

In [11]:
### Load IIDs
### Load Metafile from D. Reich:
def load_eigenstrat_anno(path="./Data/ReichLabEigenstrat/Raw/v37.2.1240K.clean4.anno", anc_only=True):
    """Load annotated Eigenstrat (from D. Reich's group)"""
    df_anno = pd.read_csv(path, sep="\t", engine="python")
    coverage = pd.to_numeric(df_anno["Coverage"], errors='coerce')
    df_anno["coverage"]=coverage

    # Convert the Ages as well
    ages = df_anno["Average of 95.4% date range in calBP (defined as 1950 CE)  "]
    df_anno["ages"] = pd.to_numeric(ages, errors='coerce')  #

    ### Convert Longitude and Latitude
    lat = df_anno["Lat."]
    lon = df_anno["Long."]
    df_anno["lat"] = pd.to_numeric(lat, errors='coerce')
    df_anno["lon"] = pd.to_numeric(lon, errors='coerce')
    df_anno["iid"] = df_anno["Instance ID"]
    
    df_anc = df_anno[df_anno["ages"]>0]

    print(f"Loaded {len(df_anc)} / {len(df_anno)} ancient Indivdiuals.")
    print(f"Without Coverage: {np.sum(np.isnan(coverage))}")
    if anc_only:
        df_anno=df_anc
    return df_anno

In [12]:
df_anno = load_eigenstrat_anno()
df_ana = df_anno[df_anno["coverage"]>0.5]
print(len(df_ana))
df_ana = df_ana[:10]  # how many individuals to extract

### Create Paths
paths = give_iid_paths(df_ana["iid"], base_folder="./Empirical/Eigenstrat/Reichall/", suffix='_roh_full.csv')

Loaded 2106 / 5081 ancient Indivdiuals.
Without Coverage: 2581
1099


### Create Paths for rerun

In [13]:
### Check if the Paths are actually there (stand alone!)
def create_rerun_csv(meta_path, base_folder, suffix="_roh_full.csv", save_path="", min_cov=0.5):
    """Ceck for Paths that have not been created (from meta in meta_path, files in base_folder with suffix)
    and save """
    df_anno = pd.read_csv(meta_path)
    df_ana = df_anno[df_anno["mean_cov"]>0.5]

    ### Create Paths
    paths = give_iid_paths(df_ana["iid"], base_folder=base_folder, suffix=suffix)

    not_there = []
    idcs = []

    for i, p in enumerate(paths):
        if not os.path.exists(p):
            not_there.append(p)
            idcs.append(i)

    print(f"Did not find {len(not_there)} / {len(paths)} paths")
    
    df_rerun = pd.DataFrame({"iid":df_ana["iid"].values[idcs], "coverage":2.00})
    if len(save_path)>0:
        df_rerun.to_csv(save_path, index=False)
        print(f"Saved {len(df_rerun)} rerun iids to {save_path}!")
    return df_rerun

In [14]:
df_rerun = create_rerun_csv(meta_path="./Data/ReichLabEigenstrat/Raw/meta.csv", base_folder="./Empirical/Eigenstrat/Reichall/", 
                 save_path="")

Did not find 0 / 1099 paths


In [6]:
df_rerun

iid  coverage
0  IPY10.SG       2.0

In [11]:
df_rerun = create_rerun_csv(meta_path="./Data/ReichLabEigenstrat/Raw/meta.csv", base_folder="./Empirical/Eigenstrat/Reichall/", 
                 save_path="./PackagesSupport/cluster_runs/ES_callROH/rerun.csv")

Did not find 235 / 1099 paths
Saved 235 rerun iids to ./PackagesSupport/cluster_runs/ES_callROH/rerun.csv!


In [15]:
### Load to test whether everything has been loaded
df_test = pd.read_csv("./PackagesSupport/cluster_runs/ES_callROH/rerun.csv")

## Create Metafile for the top 100 IIDs to rerun 
Idea: Save these Posteriors for more detailed analysis

In [15]:
save_path="./PackagesSupport/cluster_runs/ES_callROH/rerun_top100.csv"

df1 = pd.read_csv("./Empirical/Eigenstrat/Reichall/combined_roh.csv", '\t')
print(f"Loaded {len(df1)} Individuals")

df_rerun = pd.DataFrame({"iid":df1["iid"].values[:100], "coverage":2.00})   
### AAAAnd save
df_rerun.to_csv(save_path, index=False)
print(f"Saved {len(df_rerun)} rerun iids to {save_path}!")

Loaded 1098 Individuals
Saved 100 rerun iids to ./PackagesSupport/cluster_runs/ES_callROH/rerun_top100.csv!


In [16]:
df_rerun.head(10)

iid  coverage
0  Loschbour_published.DG       2.0
1      Loschbour_snpAD.DG       2.0
2  Stuttgart_published.DG       2.0
3       UstIshim_snpAD.DG       2.0
4  Ust_Ishim_published.DG       2.0
5        Vindija_snpAD.DG       2.0
6      Altai_published.DG       2.0
7          Altai_snpAD.DG       2.0
8          Goyet_final.SG       2.0
9     Les_Cottes_final.SG       2.0

### Load ROH data from Individuals and combine

In [2]:
meta_path="./Data/ReichLabEigenstrat/Raw/meta.csv"
df_anno = pd.read_csv(meta_path)
df_ana = df_anno[df_anno["mean_cov"]>0.5]
print(len(df_ana))
df_ana = df_ana[:20]  # how many individuals to extract
iids = df_ana["iid"]

1099


In [3]:
df1 = pp_individual_roh(iids, meta_path="./Data/ReichLabEigenstrat/Raw/meta.csv", base_folder="./Empirical/Eigenstrat/Reichall/",
                        save_path="./Empirical/Eigenstrat/Reichall/combined_roh_test.csv", output=False)

Loaded 20 / 2106 Individuals from Meta
Saved to: ./Empirical/Eigenstrat/Reichall/combined_roh_test.csv


# Post-Process Marcus ancients
Two folders: One with readcounts ("./Empirical/1240k/MarcusAncs/") and one with 
pseudohaploid mode ("./Empirical/1240k/MarcusAncsPH/")

### 1) Get the relevant IIDs

In [5]:
meta_path = "./Data/Marcus2019_1240k/meta_rev_final.csv"
min_cov=0.5
anc_sardind= 85
anc_ind =  1087
meta_df = pd.read_csv(meta_path)
df_anno = meta_df[anc_sardind:anc_ind]
print(f"Loaded {len(df_anno)} Ancients")

df1 = df_anno[(df_anno["mean_cov"] > min_cov) & (df_anno["include_alt"] > 0)]
iids = df1["iid"]
print(f"Nr. high coverage ancients: {len(iids)}")

Loaded 1002 Ancients
Nr. high coverage ancients: 517


### 2) Create the combined Result File

In [8]:
%%time
df1 = pp_individual_roh(iids, meta_path="./Data/Marcus2019_1240k/meta_rev_final.csv", base_folder="./Empirical/1240k/MarcusAncsPH/",
                        save_path="./Empirical/1240k/MarcusAncsPH/combined_roh.csv", output=False)

Loaded 517 / 4616 Individuals from Meta
Saved to: ./Empirical/1240k/MarcusAncsPH/combined_roh.csv
CPU times: user 4min 53s, sys: 211 ms, total: 4min 53s
Wall time: 4min 58s


# Area 51

In [7]:
df1

iid    max_roh     sum_roh  n_roh                   label        lat  \
0    I2521  69.137798  348.868188     17       Balkans_Neolithic  43.160890   
1    I1131  40.142190  297.774458     20       Balkans_Neolithic  44.900000   
2    I4303  39.335495  271.129694     20              France_MLN  43.457000   
3    I2606  38.529903  270.081087     19               England_N  51.492750   
4    I4626  25.221997  259.077697     41               Latvia_HG  56.283300   
5    I4627  38.893300  253.566578     28               Latvia_MN  56.283300   
6    I4916  46.009802  251.708633     25           Iron_Gates_HG  44.640262   
7    I9006  62.176999  223.788018     14               Mycenaean  37.970000   
8    I4068  58.320597  214.821783     15  Beaker The Netherlands  52.733563   
9    I9005  57.713205  191.249618     15                  Minoan  35.080000   
10   I4435  25.390297  179.599818     21               Latvia_MN  56.283300   
11   I4632  44.143700  176.241395     20               Latvia_HG  56.283300   
12   I5244  19.869298  173.365199     23           Iron_Gates_HG  44.595879   
13   I2534  32.640696  161.329506     20              Romania_HG  44.023000   
14   I2981  24.178701  152.726799     16         Scotland_CA_EBA  59.281000   
15   I5236  40.362095  145.688188     14           Iron_Gates_HG  44.595879   
16   I4111  48.379105  138.709307     15       Ukraine_Neolithic  48.914220   
17   I4439  12.983197  137.788184     22               Latvia_HG  56.283300   
18   I5408   8.389601  133.456879     26           Iron_Gates_HG  44.517000   
19   I5401  11.578107  133.149014     24           Iron_Gates_HG  44.640262   
20   I3948  36.215400  132.255602     10       Balkans_Neolithic  43.589000   
21   I2597  42.673499  128.961100      6          England_CA_EBA  51.160000   
22   I2457  33.378006  128.593104     10          England_CA_EBA  51.160000   
23   I4550   9.960199  126.645616     22               Latvia_HG  56.283300   
24   I4437  18.646401  123.696993     18               Latvia_MN  56.283300   
25   I1495  33.296399  121.276093     10               Europe_EN  47.167000   
26   I1763  12.226296  119.378504     19      Ukraine_Mesolithic  48.300000   
27   I0410  32.219502  118.461412     15               Europe_EN  42.500000   
28   I5772  10.447599  114.143880     19           Iron_Gates_HG  44.530000   
29   I2937  47.355098  111.874387     10   Peloponnese_Neolithic  36.638333   
..     ...        ...         ...    ...                     ...        ...   
470  I0744   0.000000    0.000000      0              Anatolia_N  40.300000   
471  I6582   0.000000    0.000000      0   Beaker Central Europe  50.090000   
472  I2683   0.000000    0.000000      0      Anatolia_BronzeAge  37.917740   
473  I7212   0.000000    0.000000      0   Beaker Central Europe  50.410265   
474  I2454   0.000000    0.000000      0          Beaker Britain  52.340000   
475  I0745   0.000000    0.000000      0              Anatolia_N  40.300000   
476  I2741   0.000000    0.000000      0   Beaker Central Europe  47.383224   
477  I1665   0.000000    0.000000      0                Iran_ChL  34.500000   
478  I7573   0.000000    0.000000      0             England_MBA  52.310608   
479  I4896   0.000000    0.000000      0   Beaker Central Europe  50.050415   
480  I0046   0.000000    0.000000      0               Europe_EN  51.890000   
481  I7576   0.000000    0.000000      0             England_MBA  52.122402   
482  I4331   0.000000    0.000000      0       Balkans_BronzeAge  43.194000   
483  I4089   0.000000    0.000000      0    Balkans_Chalcolithic  47.725300   
484  I0059   0.000000    0.000000      0             Europe_LNBA  51.820000   
485  I4229   0.000000    0.000000      0           Beaker Iberia  39.090000   
486  I3920   0.000000    0.000000      0   Peloponnese_Neolithic  36.638333   
487  I2369   0.000000    0.000000      0               Baden_LCA  47.620940   
488  I3874   0.000000    0.000000      0  Beaker Southern